This code uses built-in functions from the Image class in PIL to execute isometries on a given image. It snapshots the lens view after each isometry, then puts those images into a numpy array. It then calculates the persistent homology of the data. For different images, 'lens_sz' and 'intervals' should be adjusted.

In [1]:
# Imports
import numpy as np
import math
from hausdorff import hausdorff_distance

# For image and video manipulation
from PIL import Image
from PIL import ImageEnhance
from PIL import ImageFilter
import PIL
import glob

# Persistent homology
from ripser import ripser
from persim import plot_diagrams
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
# GHUDI stuff
from gudhi.cover_complex import MapperComplex, GraphInducedComplex, NerveComplex
from gudhi import bottleneck_distance
import gudhi as gd
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Grab image from files
c1_names = glob.glob('../images/*C1*.png')
c4_names = glob.glob('../images/*C4*.png')
print(c1_names)
print(c4_names)
img = Image.open(c1_names[5])
img = img.convert('L')

img2 = Image.open(c1_names[5])
img2 = img2.convert('L')
img2 = img2.resize((3809*3,1614*3), resample = PIL.Image.Resampling.BOX)

imgSmallArrows = Image.open(c1_names[4]).convert('L')
img4 = Image.open(c4_names[0])
img4 = img4.convert('L')

#img = img.resize((540,331), resample = PIL.Image.Resampling.BOX)
#img = ImageEnhance.Sharpness(img.convert('RGB'))
#img = img.enhance(0.1)
#img = img.filter(ImageFilter.GaussianBlur(1))
# with np.printoptions(threshold=np.inf):
#     print(np.asarray(img))
#img.show()

['../images\\C1 curly arrow.png', '../images\\C1 horsehex.png', '../images\\C1 horseshoe.png', '../images\\C1 spiral.png', '../images\\C1arrows.png', '../images\\C1bigarrows.png']
['../images\\C4pinwheel.png']


In [3]:
# wrapper for hausdorff so we can feed it into ripser
def hausdorff_wrapper(lensSize, metric='manhattan'):
    def second_layer(arrayOne, arrayTwo):
        arrayOne = arrayOne.reshape((lensSize,lensSize))
        arrayTwo = arrayTwo.reshape((lensSize,lensSize))
        return hausdorff_distance(arrayOne, arrayTwo, distance=metric)
    return second_layer

In [4]:
# if you want to store generated images in a file
def store_snapshots(image_ls):
    counter = 0
    for img1 in image_ls:
        img1.save("../images/dataset/"+str(counter)+".png")
        counter+=1

In [5]:
# see how many points are persisting
def persistingPoints(res,simpDim = 1):
    Hn = res['dgms'][simpDim].tolist() # all intervals in simpDim
    d = []
    for pair in Hn: 
        d.append(pair[1] - pair[0]) # finding lengths of intervals in Hn
    d.sort(reverse = True) # sorting from longest to shortest intervals
    print(d[:6])

translation_snapshot() translates the plane by a vector a and returns the lens view afterwards. It takes an image (Image), the lens size (int), and the translation vector (tuple) that shifts the plane in the direction of the vector. 

In [6]:
def translation_snapshot(img, lens_sz, a):
    (x,y)=a
    x=-x #since x and y will shift the crop and not the plane, need to make sure it is going in right direction
    (length, height) = img.size
    lens_corner_x=length/2.0 - lens_sz/2.0 +x
    lens_corner_y=height/2.0 - lens_sz/2.0 +y
    ret_img = img.crop((lens_corner_x,lens_corner_y, lens_corner_x+lens_sz, lens_corner_y+lens_sz))
    
    #make sure didn't fall off of the image
    (ret_length, ret_height) = ret_img.size
    if (lens_corner_x<0 or lens_corner_y<0 or lens_corner_x+lens_sz>length or lens_corner_y+lens_sz>height):
        print("I came off the page!")
        return False
    return ret_img

In [7]:
def fixedIso(img, lens_sz, degree_num, translation_num, zero_translations, lens_crunch = False,rng_seed = 451):
    
    image_list = []
    rng = np.random.default_rng(rng_seed)
    
    lens_range = ((min(img.size)/math.sqrt(2))-lens_sz)/2.0
    degrees = np.linspace(0, 360, degree_num)
    intervals_x = np.linspace((-1)*lens_range, lens_range, translation_num)
    intervals_y = np.linspace((-1)*lens_range, lens_range, translation_num)
    
    for d in degrees:
        rot = img.rotate(d,resample = PIL.Image.Resampling.NEAREST)
        #print(d)
        for x in intervals_x:
            for y in intervals_y:
                if lens_crunch == False:
                        image_list.append(translation_snapshot(rot, lens_sz, (x,y)))
                else:
                        image_list.append(translation_snapshot(rot, lens_sz, (x,y)).resize((math.ceil(lens_sz / crunch_kernel),
                                                                                          math.ceil(lens_sz / crunch_kernel)),
                                                                                         resample = PIL.Image.Resampling.BOX))

    for j in range(zero_translations):
        x = rng.integers((-1)*lens_range, lens_range)
        y = rng.integers((-1)*lens_range, lens_range)
        if lens_crunch == False:
                    image_list.append(translation_snapshot(img, lens_sz, (x,y)))
        else:
                    image_list.append(translation_snapshot(img, lens_sz, (x,y)).resize((math.ceil(lens_sz / crunch_kernel),
                                                                                      math.ceil(lens_sz / crunch_kernel)),
                                                                                     resample = PIL.Image.Resampling.BOX))
    return image_list

In [8]:
def transIso(img, lens_sz, degree_num, translation_num, zero_translations, lens_crunch = False,rng_seed = 451):
    
    image_list = []
    rng = np.random.default_rng(rng_seed)
    
    lens_range = ((min(img.size)/math.sqrt(2))-lens_sz)/2.0
    degrees = np.linspace(0, 360, degree_num)
    
    for d in degrees:
        rot = img.rotate(d,resample = PIL.Image.Resampling.NEAREST)
        #print(d)
        for j in range(translation_num):
            x = rng.integers((-1)*lens_range, lens_range)
            y = rng.integers((-1)*lens_range, lens_range)
            if lens_crunch == False:
                    image_list.append(translation_snapshot(rot, lens_sz, (x,y)))
            else:
                    image_list.append(translation_snapshot(rot, lens_sz, (x,y)).resize((math.ceil(lens_sz / crunch_kernel),
                                                                                      math.ceil(lens_sz / crunch_kernel)),
                                                                                     resample = PIL.Image.Resampling.BOX))

    for j in range(zero_translations):
        x = rng.integers((-1)*lens_range, lens_range)
        y = rng.integers((-1)*lens_range, lens_range)
        if lens_crunch == False:
                    image_list.append(translation_snapshot(img, lens_sz, (x,y)))
        else:
                    image_list.append(translation_snapshot(img, lens_sz, (x,y)).resize((math.ceil(lens_sz / crunch_kernel),
                                                                                      math.ceil(lens_sz / crunch_kernel)),
                                                                                     resample = PIL.Image.Resampling.BOX))
    return image_list

In [9]:
def randIso(img, lens_sz, degree_num, translation_num, zero_translations, lens_crunch = False,rng_seed = 451):
    
    image_list = []
    rng = np.random.default_rng(rng_seed)
    
    lens_range = ((min(img.size)/math.sqrt(2))-lens_sz)/2.0
    degrees = rng.uniform(0.0,360.0, size = degree_num)
    j = 0
    
    for d in degrees:
        #print(j)
        j = j + 1
        rot = img.rotate(d,resample = PIL.Image.Resampling.NEAREST)
        x = rng.integers((-1)*lens_range, lens_range)
        y = rng.integers((-1)*lens_range, lens_range)
        if lens_crunch == False:
                    image_list.append(translation_snapshot(rot, lens_sz, (x,y)))
        else:
                    image_list.append(translation_snapshot(rot, lens_sz, (x,y)).resize((math.ceil(lens_sz / crunch_kernel),
                                                                                      math.ceil(lens_sz / crunch_kernel)),
                                                                                     resample = PIL.Image.Resampling.BOX))
    for k in range(zero_translations):
        x = rng.integers((-1)*lens_range, lens_range)
        y = rng.integers((-1)*lens_range, lens_range)
        if lens_crunch == False:
                    image_list.append(translation_snapshot(img, lens_sz, (x,y)))
        else:
                    image_list.append(translation_snapshot(img, lens_sz, (x,y)).resize((math.ceil(lens_sz / crunch_kernel),
                                                                                      math.ceil(lens_sz / crunch_kernel)),
                                                                                     resample = PIL.Image.Resampling.BOX))
    return image_list

In [10]:
def imgListToArray(img_list):
    data = []
    x=0
    for i in image_list:
        img_arr = np.asarray(i)
        data.append(img_arr.reshape(-1))
    data = np.array(data)
    return data

In [26]:
image_list = transIso(imgSmallArrows, lens_sz = 16, degree_num = 12, translation_num = 3, zero_translations = 20)
data = imgListToArray(image_list)

In [27]:
tc = gd.TangentialComplex(intrisic_dim = 3,points= data)
tc.compute_tangential_complex()
result_str = 'Tangential contains ' + repr(tc.num_vertices()) + ' vertices.'
print(result_str)

if tc.num_inconsistent_simplices() > 0:
    print('Tangential contains inconsistencies.')
    
tc.fix_inconsistencies_using_perturbation(10, 60)
if tc.num_inconsistent_simplices() == 0:
    print('Inconsistencies has been fixed.')

Tangential contains 56 vertices.
Tangential contains inconsistencies.


In [28]:
st = tc.create_simplex_tree()
result_str = 'Simplex tree is of dimension ' + repr(st.dimension()) + \
    ' - ' + repr(st.num_simplices()) + ' simplices - ' + \
    repr(st.num_vertices()) + ' vertices.'
print(result_str)
for filtered_value in st.get_filtration():
    print(filtered_value[0])

Simplex tree is of dimension 3 - 2755 simplices - 56 vertices.
[0]
[1]
[0, 1]
[2]
[3]
[4]
[3, 4]
[5]
[0, 5]
[1, 5]
[0, 1, 5]
[6]
[3, 6]
[4, 6]
[7]
[3, 7]
[6, 7]
[8]
[0, 8]
[1, 8]
[0, 1, 8]
[5, 8]
[0, 5, 8]
[1, 5, 8]
[0, 1, 5, 8]
[9]
[2, 9]
[7, 9]
[10]
[4, 10]
[6, 10]
[7, 10]
[11]
[3, 11]
[4, 11]
[6, 11]
[10, 11]
[4, 10, 11]
[12]
[2, 12]
[8, 12]
[9, 12]
[2, 9, 12]
[13]
[3, 13]
[4, 13]
[3, 4, 13]
[6, 13]
[4, 6, 13]
[10, 13]
[4, 10, 13]
[6, 10, 13]
[11, 13]
[4, 11, 13]
[6, 11, 13]
[14]
[4, 14]
[6, 14]
[10, 14]
[11, 14]
[4, 11, 14]
[13, 14]
[4, 13, 14]
[6, 13, 14]
[15]
[0, 15]
[1, 15]
[0, 1, 15]
[5, 15]
[0, 5, 15]
[1, 5, 15]
[0, 1, 5, 15]
[8, 15]
[0, 8, 15]
[5, 8, 15]
[0, 5, 8, 15]
[16]
[3, 16]
[4, 16]
[6, 16]
[7, 16]
[10, 16]
[6, 10, 16]
[11, 16]
[4, 11, 16]
[13, 16]
[4, 13, 16]
[11, 13, 16]
[4, 11, 13, 16]
[17]
[2, 17]
[7, 17]
[8, 17]
[9, 17]
[2, 9, 17]
[7, 9, 17]
[12, 17]
[8, 12, 17]
[9, 12, 17]
[18]
[2, 18]
[7, 18]
[9, 18]
[2, 9, 18]
[7, 9, 18]
[12, 18]
[2, 12, 18]
[9, 12, 18]
[2, 9, 1

In [29]:
st.persistence()

st.betti_numbers()


[1, 0, 26]